In [1]:
import copy
import csv

import requests
import logging


# set number of laptops on one page
PAGE_SIZE = 20
# set max page or parse all pages with float('inf')
MAX_PAGE = 1

cookies = {
    'MVID_CITY_ID': 'CityCZ_975',
    'MVID_REGION_ID': '1',
    'MVID_REGION_SHOP': 'S002',
    'MVID_TIMEZONE_OFFSET': '3',
}

GET_LAPTOPS_IDS_URL = (
    'https://www.mvideo.ru/bff/products/listing'
    '?categoryId=118'
    '&offset={}'
    '&limit={}'
    '&filterParams=WyJ0b2xrby12LW5hbGljaGlpIiwiLTEyIiwiZGEiXQ%3D%3D'
    '&doTranslit=true'
)
GET_LAPTOPS_DETAILS_URL = 'https://www.mvideo.ru/bff/product-details/list'
GET_LAPTOP_DETAILS_JSON_BODY = {
    "productIds": [],
    "mediaTypes": ["images"],
    "category": True,
    "status": True,
    "brand": True,
    "propertyTypes": ["KEY"],
    "propertiesConfig": {"propertiesPortionSize": 5},
    "multioffer": False
}
GET_LAPTOPS_PRICES_URL = (
    'https://www.mvideo.ru/bff/products/prices'
    '?productIds={}'
    '&addBonusRubles=true'
    '&isPromoApplied=true'
)


def main():
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger()
    parsed_info = {}
    PAGE_COUNTER = 0
    while PAGE_COUNTER < MAX_PAGE:
        logger.info(f'started parsing laptops page №{PAGE_COUNTER}')

        # get page <PAGE_COUNTER> laptop ids
        laptops_ids = requests.get(
            GET_LAPTOPS_IDS_URL.format(PAGE_COUNTER * PAGE_SIZE, PAGE_SIZE),
            headers={'User-Agent': 'Mozilla/5.0 (U; Linux x86_64) Gecko/20100101 Firefox/69.6'},
            cookies=cookies,
        ).json()['body']['products']
        logger.info('laptops ids parsed')

        # get all laptop details
        json_body = copy.deepcopy(GET_LAPTOP_DETAILS_JSON_BODY)
        json_body['productIds'] = laptops_ids
        laptops_details = requests.post(
            GET_LAPTOPS_DETAILS_URL,
            json=json_body,
            headers={
                'referer': 'https://www.mvideo.ru/noutbuki-planshety-komputery-8/noutbuki-118?from=under_search',
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
            },
            cookies=cookies,
        ).json()
        logger.info('laptops details parsed')

        # parse collected laptops details
        for laptop in laptops_details['body']['products']:
            parsed_info[laptop['productId']] = {
                'id': laptop['productId'],
                'name': laptop['name'],
                'rating': laptop['rating']['star'],
                'review_count': laptop['rating']['count']
            }

        # Get all laptop prices
        laptops_prices = requests.get(
            GET_LAPTOPS_PRICES_URL.format(','.join(laptops_ids)),
            headers={
                'referer': 'https://www.mvideo.ru/noutbuki-planshety-komputery-8/noutbuki-118?from=under_search',
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
            },
            cookies=cookies,
        ).json()
        logger.info('laptops prices parsed')

        #parse collected laptops prices
        for laptop in laptops_prices['body']['materialPrices']:
            parsed_info[laptop['productId']]['base_price'] = laptop['price']['basePrice']
            parsed_info[laptop['productId']]['sale_price'] = laptop['price']['salePrice']

        PAGE_COUNTER += 1

    # after parsing all pages write to csv
    with open('parsed.csv', 'w') as f:  # You will need 'wb' mode in Python 2.x
        parsed_laptops = list(parsed_info.values())
        print(list(parsed_laptops[0].keys()))
        w = csv.DictWriter(f, list(parsed_laptops[0].keys()))
        w.writeheader()
        for laptop in parsed_laptops:
            print(laptop)
            w.writerow(laptop)

if __name__ == '__main__':
    main()


INFO:root:started parsing laptops page №0
INFO:root:laptops ids parsed
INFO:root:laptops details parsed
INFO:root:laptops prices parsed


['id', 'name', 'rating', 'review_count', 'base_price', 'sale_price']
{'id': '400235054', 'name': 'Ноутбук игровой Thunderobot 911S Core D/15.6"/Core i5-12450H/16/512/RTX 3050/noOS/Black', 'rating': 4.816993464052287, 'review_count': 306, 'base_price': 94999, 'sale_price': 84999}
{'id': '30069844', 'name': 'Ноутбук Lenovo V15 G4 AMN/15.6"/AMD Ryzen 3 7320U/8/512/noOS/Grey', 'rating': 4.914893617021277, 'review_count': 47, 'base_price': 49999, 'sale_price': 39999}
{'id': '30069883', 'name': 'Ноутбук HUAWEI MateBook D 15 2021 BoD-WDH9/15.6"/AMD Ryzen 7 5700U/8/512/Win/Space Gray', 'rating': 4.9245283018867925, 'review_count': 53, 'base_price': 68999, 'sale_price': 59999}
{'id': '30070140', 'name': 'Ноутбук игровой MSI Katana 17/17.3"/Core i5-12450H/8/512/RTX 2050/noOS/Black', 'rating': 4.891304347826087, 'review_count': 46, 'base_price': 94999, 'sale_price': 77999}
{'id': '30070422', 'name': 'Ноутбук HUAWEI MateBook D 16 2024/16"/Core i5-12450H/16/1TB/Win/Space Gray', 'rating': 4.89393939